In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from os import path
import json


In [2]:
def normalize(text, language='english'):
    tokens = word_tokenize(text)
    stop = set(stopwords.words(language))
    tokens = [token for token in tokens if token not in stop]
    tokens = [token for token in tokens if token.isalnum()]
    tokens = lemmatize(tokens)
    return tokens


In [3]:
def tokenize(text, language='english'):
    stop = set(stopwords.words(language))
    return [token for token in word_tokenize(text) if token not in stop and token.isalnum()]


In [4]:
def stem(tokens, language='english'):
    stemmer = SnowballStemmer(language)
    return [stemmer.stem(token) for token in tokens]


In [5]:
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]


In [11]:
def tokenizeFile(targetFile, language='english'):
    targetFileName = path.basename(targetFile)
    destinationFileName = "tokenized." + targetFileName
    with open(targetFile, encoding='utf-8') as fin, open(destinationFileName, 'w', newline='') as fout:
        for row in fin:
            doc = json.loads(row)
            content = doc['content']
            doc['content'] = " ".join(normalize(content))
            print(json.dumps(doc), file=fout)


In [12]:
source = '..\\..\\datasets\\raw.dump.jsonl'
tokenizeFile(source)